In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import torch
from torchdiffeq import odeint
from physvae import utils
from physvae.locomotion.model import VAE, PlanarVAE

In [ ]:
# load data
datadir = './data'
dataname = 'test'
data_test = sio.loadmat('{}/data_{}.mat'.format(datadir, dataname))['data'].astype(np.float32)
print(data_test.shape)
_, dim_x, dim_t = data_test.shape

In [ ]:
method = ['nnonly', 'physonly', 'physnn', 'nf', 'attnf']
idx=0
dat = data_test[idx].T
plt.figure()
plt.subplot(3,1,1)
plt.plot(dat[:,0])
plt.subplot(3,1,2)
plt.plot(dat[:,1])
plt.subplot(3,1,3)
plt.plot(dat[:,2])

for m in method:
    # load training args as dict
    dim_y = 3
    modeldir = './out_locomotion'

    with open('{}/args_{}.json'.format(modeldir,m), 'r') as f:
        args_tr_dict = json.load(f)

    # set model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = PlanarVAE(args_tr_dict).to(device)

    # set model
    if args_tr_dict.flow == 'no_flow':
        model = VAE(vars(args_tr_dict)).to(device)
    elif args_tr_dict.flow == 'planar':
        model = PlanarVAE(vars(args_tr_dict)).to(device)

    # load model
    model.load_state_dict(torch.load('{}/model_{}.pt'.format(modeldir,m), map_location=device))
    print('model {} loaded'.format(m))
    model.eval()
    data_test_tensor = torch.Tensor(data_test).to(device).contiguous()
    z_phy_stat, z_aux2_stat, x, _ = model(data_test_tensor)
    # plot
    plt.subplot(3,1,1)
    plt.plot(x[idx][:,0].detach().cpu().numpy().T, 'k--')
    plt.subplot(3,1,2)
    plt.plot(x[idx][:,1].detach().cpu().numpy().T, 'k--')
    plt.subplot(3,1,3)
    plt.plot(x[idx][:,2].detach().cpu().numpy().T, 'k--')


plt.savefig('{}/recon.png'.format(args_tr_dict['outdir']))
plt.show()